In [1]:
import pandas as pd

dataframes = pd.read_pickle('data/review.pkl')

# 기존에저장한 stores / reviews 데이터를 merge합니다. (reviews에 category 관련 정보 저장용)
category_score = pd.merge(
    dataframes["stores"], dataframes["reviews"], left_on="id", right_on="store"
)[["category", "user", "score"]]

# category를 기준으로 각각의 row를 여러개의 row로 분리했습니다.
category_score["category"] = category_score["category"].apply(lambda c: c.split("|"))
category_explode = category_score.explode("category").set_index('user')

# category_explode의 columns : [user, category, score]
# 현재는 한 명의 user가 여러개의 category에 중복해서 점수를 준 상황입니다.
# (서로 다른 음식점이 같은 category를 가진 경우)

In [2]:
# category_explode에 있는 데이터를 category의 평균 평점 / 리뷰 개수로 바꿔주었습니다.
category_scores = (category_explode.groupby(["category"])["score"]
                  .agg(["mean", "size"])
                  .reset_index()
                  .drop([0]))

# category_scores 리뷰의 개수가 5개 미만인 category를 제거했습니다.
# (추천 정확도 향상 목적 / 추천할 때 죄다 평점이 5.0이어서 추천의 의미가 없음)
category_scores_mean = category_scores[category_scores["size"] >= 5].set_index(["category"])["mean"].index

In [3]:
# category_user_score : 기존 category_explode에서 리뷰 수가 5개 이상인 리뷰만 남겼습니다.
category_user_score = category_explode[category_explode["category"].isin(category_scores_mean)].reset_index()

# 다시 user / category의 평점을 기준으로 평균값을 계산했고,
category_user_score = category_user_score.groupby(["user","category"])["score"].mean().reset_index()

# user-category의 full matrix를 생성했습니다. (score_matrix)
score_matrix = pd.pivot(data=category_user_score, index="user", columns="category", values="score")

In [4]:
def recommend_category(user, n_items):
    """
    각각의 user에 대해 best_seller model을 이용해서 n_items 개의 카테고리를 추천해줍니다.
    user_category : 실제 데이터를 기준으로 카테고리에 대한 현재 유저의 평가 정보 데이터를 가져옵니다.
    rating_matrix에 있는 category들에 대해서
    만약 user_category의 해당 category 값이 NaN이라면, 유저가 해당 카테고리를 평가하지 않았다는 의미입니다.
    해당 경우에 best_seller model을 사용해서 예상값을 입력했습니다.
    아닐 경우 user가 입력한 category의 평균 평점을 유지했습니다.
    예상값을 모두 입력한 후에는 정렬 후 n_items개의 Item만 return합니다.
    """
    user_category = score_matrix.loc[user]
    for category in score_matrix:
        if pd.isnull(user_category.loc[category]):
            user_category.loc[category] = best_seller(user, category)
    print(user_category)
    category_sort = user_category.sort_values(ascending=False)[:n_items]
    return category_sort

# 전체 평균으로 예측치를 계산하는 기본 모델
def best_seller(user, category):
    """
        category_mean dataframe에 category가 있다면 (평가 데이터가 존재하는 경우)
        해당 평균값을 리턴합니다.
        아닌 경우, 전체 리뷰의 평점을 리턴합니다. (중간값보다는 평균값을 채용했습니다.)
    """
    try:
        score = category_mean[category]
    except:
        score = category_explode['score'].mean()
    return score

# category_mean : 각 카테고리의 전체 평균 평점입니다.
category_mean = category_explode.groupby(['category'])['score'].mean()
print(recommend_category(7, 10))
# 최초에는 5.0의 평점을 가진 카테고리가 나오는데,
# 이건 사용자가 5점을 준 리뷰들이 있어서 이렇게 나옵니다.
# 5.0 이후에는 전체 평점이 높은 카테고리가 출력됩니다.

category
bar       4.090909
cafe      3.806283
la갈비      3.545455
lp바       4.400000
pasta     3.886364
            ...   
흑염소       3.526316
흑우        3.500000
흑임자팥빙수    3.214286
히레까스      4.040000
히츠마부시     4.272727
Name: 7, Length: 2385, dtype: float64
category
해산물뷔페     5.000000
철판제육볶음    5.000000
국수        5.000000
소고기국수     5.000000
떡볶이돈까스    5.000000
고려음식      4.863636
칠면조       4.857143
오리초밥      4.833333
루프트탑      4.833333
문화카페      4.833333
Name: 7, dtype: float64


In [5]:
# user의 데이터(성별, 연령)을 기준으로 그룹화하여 추천하기 위한 함수입니다.
# 먼저, users dataframes에서 유저 정보를 받아옵니다.
users = dataframes['users'].rename(columns={'id': 'user'}).set_index('user')

# category_explode와 users dataframe을 합쳐줍니다. (평가에 유저 정보 포함하기 위함)
merged_scores = pd.merge(category_explode, users, left_index=True, right_index=True)

# merge한 데이터를 category와 gender를 기준으로 점수를 통계냅니다.
# drop([0, 1])은 빈 카테고리값이 들어가서...
gender_scores = (merged_scores.groupby(['category', 'gender'])['score']
                 .agg(["size", "mean"])
                 .reset_index()
                 .drop([0, 1]))

# 마찬가지로 평가가 5개 이상인 그룹의 데이터만 남겨줬습니다.
gender_scores = gender_scores[gender_scores['size'] >= 5]
gender_scores = gender_scores.set_index(['category', 'gender'])
category_mean = category_explode.groupby(['category'])['score'].mean()
# 현재 category / gender로 계층적 인덱스가 적용되어있고, size와 mean값이 들어 있습니다.
print(gender_scores)

# 해당하는 category들만 모아줍니다.
gender_scores_index = gender_scores.index.get_level_values(0)

# 한번 cafe category의 세부 내용을 봤습니다...
# print(gender_scores.loc['cafe'])

                 size      mean
category gender                
bar      남         22  3.909091
         여         44  4.181818
cafe     남         67  3.537313
         여        124  3.951613
la갈비     여          7  3.571429
...               ...       ...
흑임자팥빙수   여          9  3.444444
히레까스     남         31  3.935484
         여         19  4.210526
히츠마부시    남         15  4.133333
         여          7  4.571429

[3623 rows x 2 columns]


In [6]:
# category_user_score : 기존 category_explode에서 리뷰 수가 5개 이상인 리뷰만 남겼습니다.
category_user_score = category_explode[category_explode["category"].isin(category_scores_mean)].reset_index()

# 다시 user / category의 평점을 기준으로 평균값을 계산했고,
category_user_score = category_user_score.groupby(["user","category"])["score"].mean().reset_index()

# user-category의 full matrix를 생성했습니다. (score_matrix)
score_matrix = pd.pivot(data=category_user_score, index="user", columns="category", values="score")

In [7]:
def recommend_gender_category(user, n_items):
    """
        이번에도 마찬가지로 gender 평균을 기준으로 추천해줬습니다.
    """
    user_category = score_matrix.loc[user]
    for category in score_matrix:
        if pd.isnull(user_category.loc[category]):
            user_category.loc[category] = best_seller_gender(user, category)
#     print(user_category)
    category_sort = user_category.sort_values(ascending=False)[:n_items]
    return category_sort

def best_seller_gender(user, category):
    """
        해당 카테고리에 대해 성별 데이터가 있다면 해당 데이터를
        해당 카테고리에 대해 성별 데이터가 없다면 전체 평균 데이터를 넣었습니다.
    """
    if category in gender_scores_index:
        gender = users.loc[user]['gender']
        if gender in gender_scores.loc[category]:
            gender_rating = gender_scores.loc[category, gender]
        else:
            gender_rating = category_mean[category]
    else:
        gender_rating = 3.0
    return gender_rating

# 별 차이 없어 보이네요 :)
print(recommend_gender_category(7, 10))

category
해산물뷔페      5.000000
국수         5.000000
소고기국수      5.000000
고려음식       4.863636
칠면조        4.857143
손순두부       4.833333
오리초밥       4.833333
핸드메이드      4.818182
양고기숯불구이    4.800000
은어튀김       4.800000
Name: 7, dtype: float64


In [8]:
# user의 연령을 기준으로 그룹화하여 추천하기 위한 함수입니다.
# 먼저, users dataframes에서 유저 정보를 받아옵니다.
users2 = dataframes['users'].rename(columns={'id': 'user'}).set_index('user')
users2['age'] = users2['age'].apply(lambda x: x//10)

# category_explode와 users dataframe을 합쳐줍니다. (평가에 유저 정보 포함하기 위함)
merged_scores2 = pd.merge(category_explode, users2, left_index=True, right_index=True)

# merge한 데이터를 category와 gender를 기준으로 점수를 통계냅니다.
age_scores = (merged_scores2.groupby(['category', 'age'])['score']
                 .agg(["size", "mean"])
                 .reset_index()
                 .drop([0, 1, 2, 3, 4, 5, 6]))

# 마찬가지로 평가가 5개 이상인 그룹의 데이터만 남겨줬습니다.
age_scores = age_scores[age_scores['size'] >= 5]
age_scores = age_scores.set_index(['category', 'age'])
print(age_scores)

              size      mean
category age                
bar      2      32  4.125000
         3      29  4.068966
cafe     2     105  3.923810
         3      65  3.723077
         4      15  3.266667
...            ...       ...
히레까스     2      22  4.090909
         3      22  4.045455
         4       5  3.800000
히츠마부시    2      11  4.363636
         3       7  3.857143

[4635 rows x 2 columns]


In [9]:
import numpy as np

# 실제값과 예측값의 RMSE 계산해주는 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true)- np.array(y_pred)) ** 2))

# 모델의 RMSE 계산
def score(model):
    id_pairs = zip(x_test['user'], x_test['category'])
    y_pred = np.array([model(user, category) for (user, category) in id_pairs])
    y_true = np.array(x_test['score'])
    return RMSE(y_true, y_pred)

In [10]:
# 이후부터는 모델의 정확성을 한번 평가해봤습니다.

from sklearn.model_selection import train_test_split

x = category_user_score.copy()
y = category_user_score['user']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

train_matrix = x_train.pivot(index='user', columns='category', values='score')
# 75%는 train set / 25%는 test set으로 분리했습니다.

In [11]:
# 전체 평균으로 예측치를 계산하는 기본 모델
def best_seller(user, category):
    try:
        score = train_mean[category]
    except:
        score = x_train['score'].mean()
    return score

train_mean = x_train.groupby(['category'])['score'].mean()
score(best_seller)

1.1309392190162577

In [12]:
# gender 별 평균을 예측치로 돌려주는 함수
def best_seller_gender(user, category):
    if (category in train_matrix) and (category in gender_mean):
        gender = users.loc[user]['gender']
        if gender in gender_mean[category]:
            gender_rating = gender_mean[category][gender]
        else:
            gender_rating = train_mean[category]
    else:
        gender_rating = x_train['score'].mean()
    return gender_rating

gender_mean = gender_scores['mean']
train_mean = x_train.groupby(['category'])['score'].mean()
score(best_seller_gender)

1.0976689304070308

In [13]:
# age band 별 평균을 예측치로 돌려주는 함수
def best_seller_age(user, category):
    if (category in train_matrix) and (category in age_mean):
        age = users.loc[user]['age']
        if age in age_mean[category]:
            age_rating = age_mean[category][age]
        else:
            age_rating = train_mean[category]
    else:
        age_rating = x_train['score'].mean()
    return age_rating

age_mean = age_scores['mean']
train_mean = x_train.groupby(['category'])['score'].mean()
score(best_seller_age)

1.1282858320238718